#### Lost in then middle:长上下文精度问题

In [ ]:
# 安装依赖，用于句式转换
%pip install sentence-transformers

In [ ]:
from langchain.chains import LLMChain,StuffDocumentsChain
from langchain_community.document_transformers import (
    # 长上下文记录器
    LongContextReorder
)
# 词嵌入
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
# Chroma 是一个开源的嵌入式向量数据库。它可以存储和检索向量化后的数据，非常适合用于语义搜索和相似性查找。
from langchain_community.vectorstores import Chroma

# 准备文本数据，模拟切割后结果
text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登之环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里.伯德是一位标志性的NBA球员。"
]

# 使用HuggingFace模型托管平台，使用托管的开源LLM来做词嵌入，MiniLM-L6-v2是一个较小的LLM 
embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

# 将文本向量化，并存储到 Chroma 向量数据库中
# 给定的文本列表 (texts) 和嵌入模型 (embeddings) 创建一个 Chroma 向量数据库实例
# as_retriever():这个方法将 Chroma 向量数据库实例转换为一个检索器（retriever）对象
retrieval = Chroma.from_texts(text, embeddings).as_retriever(
    search_type="mmr",
    # 返回最相关的10条结果
    search_kwargs={"k": 10, "fetch_k": 200, "lambda_mult": 0},
)

# 根据相关性返回文本块
query = "关于凯尔特人队你知道什么?"
docs = retrieval.get_relevant_documents(query)
print(docs)

for doc in docs:
    print(doc)

In [ ]:
# 对检索结果进行重新排序，根据论文的方案
# 问题相关性越低的内容块放在中间
# 问题相关性越高的内容块放在头尾
# 来解决长文本的切分后精度下降的问题

# 用于长上下文记录器，对文档进行重新排序
reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs)

# 头尾共有4个高相关性内容块
reo_docs

In [ ]:
from dotenv import load_dotenv
import os

# 主要作用是从一个名为 .env 的文件中加载环境变量到你的 Python 程序中
load_dotenv(".env")

api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

print(api_base)
print(api_key)

In [ ]:
# 检测下这种方案的精度效果
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI

#设置llm
llm = OpenAI(
    # openai_api_base=openai_api_base,
    openai_api_key=api_key,
    model="gpt-3.5-turbo-instruct",
    temperature=0
)

# 构建提示词模板
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}",
)


stuff_prompt_override ="""Given this text extracts:
----------------------------------------
{context}
----------------------------------------
Please answer the following questions:
{query}
"""

prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"]
)

# 创建一个 LLMChain 实例，它将 LLM 和提示词模板连接起来。
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# 创建一个 StuffDocumentsChain 实例，它负责将多个文档组合成一个上下文，并将其传递给 llm_chain
WorkChain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context", # 定将组合后的文档内容存储在 context 变量中。
)

# 链调用
WorkChain.run(
    input_documents=reo_docs, # 将 reo_docs 列表中的文档作为输入。
    query="我最喜欢做什么事情？"
)